In [25]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_circuit_layout, circuit_drawer
import matplotlib as plt
import numpy as np
import random
from qiskit.quantum_info import Statevector
import binascii


In [42]:
# functions

def generate_bell_pair(m):
    bell_pairs = []
    for _ in range(m):
        # create two qubits and two classical bits to store measurement results
        qc = QuantumCircuit(2, 2)
        qc.x(1)
        qc.h(0)
        qc.z(0)
        qc.z(1)
        qc.cx(0, 1)
        
        bell_pairs.append(qc)
        # print(qc)
    return bell_pairs



def select_photons_eavesdropping(m, pe=0.2):

    # calculate number of pairs to consider for eavesdropping detection
    # so, pe is probability of not selected a qubit for measurement
    num_samples = (1 - pe) * m / 2
    num_samples = max(1, int(num_samples))

    # get random sample of indices based on proportion
    selected = np.random.choice(m, num_samples, replace=False)

    return selected

# https://quantumcomputing.stackexchange.com/questions/13605/how-to-measure-in-another-basis
def apply_random_measurements(qc, indices):

    bases = {}

    for i in indices:
        basis = random.choice(['X', 'Y', 'Z'])
        bases[i] = basis

        # get bob's qubit
        q = 2 * i + 1
        if basis == 'X':
            qc.h(q)
        elif basis == 'Y':
            qc.sdg(q)
            qc.h(q)

        # qiskit measures in z basis by default
        # only measure bob's qubit
        qc.measure(q, q)

    return bases

def eavesdropping_detection(qc, indices, simulator):

    bob_bases = apply_random_measurements(qc, indices)
    bob_results = {}

    qc = transpile(qc, simulator)
    job = simulator.run(qc, shots=1)
    result = job.result().get_counts()

    result_str = list(result.keys())[0]

    for i in indices:
        bob_results[i] = result_str[2 * i + 1]

    return bob_bases, bob_results

# e.g:
    # result.get_counts()
    # {'01011000': 1}
    # bob's qubits are at 1,3,5,7


# def eavesdropping_detection(qc, indices, simulator):
#     bob_bases = {}
#     bob_results = {}

#     for i in indices:
#         # apply measurements using Bob's qubit
#         bob_bases[i] = apply_random_measurements(bell_pairs[i], 1)
#         circ = transpile(bell_pairs[i], simulator) # transpile does some sort of optimization on simulator
#         job = simulator.run(circ, shots=1) # run the quantum circuit on the simulator; shots is number of executions
#         result = job.result().get_counts()
#         # print(result)
#         # print(result.keys())

#         # result -> {'01': 1} -> observed 01 one time
#         # result.keys() returns all measured bitstrings -> dict_keys(['01'])
#         #       list(result.keys())[0][1] -> ['01'] -> '01' -> '1' (return qubit in S_B)
#         list(result.keys())[0][1]
#         bob_results[i] = list(result.keys())[0][1]

#     return bob_bases, bob_results


def error_check(qc, simulator, indices, bob_bases, bob_results, threshold=0.1):

    alice_results = {}

    for i in indices:
        # alice's qubit
        q = 2 * i
        basis = bob_bases[i]

        if basis == 'X':
            qc.h(q)
        elif basis == 'Y':
            qc.sdg(q)
            qc.h(q)

        # only measure alice's qubit
        qc.measure(q, q)

    qc = transpile(qc, simulator)
    job = simulator.run(qc, shots=1)
    result = job.result().get_counts()

    result_str = list(result.keys())[0]

    for i in indices:
        alice_results[i] = result_str[2 * i]

    print("Alice Results: ", alice_results)

    error_count = 0
    for i in indices:
        if alice_results[i] == bob_results[i]:
            error_count += 1
    
    error_rate = error_count / len(indices)

    return not (error_rate  < threshold)



# step 3 encode
def encode(bell_pairs, message):
    bin_repr = text_to_bits(message)
    for i, bit in enumerate(bin_repr):
        if bit == "1":
            bell_pairs[i].x(0)

    # sensing task?




def run_qisac(m, data, pe=0.1):

    simulator = AerSimulator()

    # Step 1: State Preparation
    bell_pairs = generate_bell_pair(m)
    print_vector_state(bell_pairs[0])
    # Step 2: First Eavesdropping Detection
    # alice "sends" bob half of the pairs (S_B)
    # bob randomly selects photons from sequence S_B
    # select_photons_eavesdropping(m)
    # pe is probability of not selected a qubit

    # Step 3: Encoding
    encode(bell_pairs, data)
    print_vector_state(bell_pairs[1])


# util functions

def get_random_indices(m, pe):
    # calculate number of pairs to consider for eavesdropping detection
    # so, pe is probability of not selected a qubit for measurement
    num_samples = (1 - pe) * m / 2
    num_samples = max(1, int(num_samples))

    print("Num samples: ", num_samples)

    # get random sample of indices based on proportion
    selected = np.random.choice(m, num_samples, replace=False)
    return selected


def print_vector_state(qc):
    # print(qc)
    state = Statevector.from_instruction(qc)
    # print(state)
    display(state.draw('latex'))

def text_to_bits(text, encoding='utf-8', errors='surrogatepass'):
    bits = bin(int(binascii.hexlify(text.encode(encoding, errors)), 16))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))

def text_from_bits(bits, encoding='utf-8', errors='surrogatepass'):
    n = int(bits, 2)
    return int2bytes(n).decode(encoding, errors)

def int2bytes(i):
    hex_string = '%x' % i
    n = len(hex_string)
    return binascii.unhexlify(hex_string.zfill(n + (n & 1)))



In [43]:
run_qisac(8, "p", 0.1)



<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [ ]:
adc_values = [3598.125, 3605.000, 3601.500]

## Step 1: Prepare states

generate m circuits where each circuit is a bell pair in state |ψ−⟩⊗m
each circuit is in a list

## Step 2: First eavesdropping

select a random subset of bits to be used for error detection
bob randomly measures these bits in a randomly selected basis
Alice measures the corresponding bits using the same bases chosen by Bob
They compare results - should get opposite results using the bell state from the paper

## Step 3: Encoding
<!-- https://docs.quantum.ibm.com/guides/simulate-with-qiskit-sdk-primitives -->




In [32]:
s = "p"
# Using a generator expression to convert each character in the string to its binary representation
b_repr = ' '.join(format(ord(char), '08b') for char in s)
print("Binary Representation:", b_repr)

Binary Representation: 01110000
